# Práctica 2: Sensado y análisis de video
## Detección de puntos de atención visual

### Librerias

In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
import cv2
import mediapipe as mp
import sys
import os
import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

C:\Users\Antonio\AppData\Local\Temp\ipykernel_1592\1506638517.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


### Segmentado y preparación de los Datos.

In [2]:
IMGS = "./Imagen/"
dir_list = os.listdir(IMGS)
dir_list.sort()
dfsCsv = []
for i in dir_list:
    fnames = os.listdir(IMGS + i)
    for f in fnames:
        if f.endswith(".csv"):
            df = pd.read_csv(IMGS + i + f"/{f}")
            name = f.replace('csv', 'mp4')
            df['source'] = IMGS + i + f"/{name}"
            df['subject'] = name[:-4]
            dfsCsv.append(df)

In [3]:
dfsCsv[1]

,Unnamed: 0,FRENTE,IZQUIERDA,DERECHA,RETROVISOR,PALANCA,source,subject
0,0,00:00 - 00:08,00:11- 00:19,00:22-00:29,00:31-00:38,00:41-00:48,./Imagen/Ariana/ARIANA.mp4,ARIANA


In [4]:
# Dataframe con formato diferente
df_last = dfsCsv[-1]
dfsCsv = dfsCsv[:-1]
df_last

,filename,inicio,fin,label,source,subject
0,SCARLETT.mp4,00:00,00:12,frente,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT
1,SCARLETT.mp4,00:13,00:24,izquierdo,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT
2,SCARLETT.mp4,00:25,00:35,central,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT
3,SCARLETT.mp4,00:37,00:47,derecho,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT
4,SCARLETT.mp4,00:48,01:00,palanca,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT


In [5]:
# Definir el mapeo de valores
mapeo_valores = {
    'frente': 'FRENTE',
    'izquierdo': 'IZQUIERDA',
    'central': 'RETROVISOR',
    'derecho': 'DERECHA',
    'palanca': 'PALANCA'
}

# Mapear los valores de la columna 'label'
df_last['label'] = df_last['label'].map(mapeo_valores)

df_last = df_last[['inicio', 'fin', 'label', 'source', 'subject']]
df_last 

,inicio,fin,label,source,subject
0,00:00,00:12,FRENTE,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT
1,00:13,00:24,IZQUIERDA,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT
2,00:25,00:35,RETROVISOR,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT
3,00:37,00:47,DERECHA,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT
4,00:48,01:00,PALANCA,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT


In [6]:
dfs_concat = []
dfs_concat.append(df_last)
for df in dfsCsv:
    source = df['source'][0]
    subject = df['subject'][0]
    columns_to_keep = ['FRENTE', 'RETROVISOR', 'IZQUIERDA', 'DERECHA', 'PALANCA']
    df = df[columns_to_keep]

    # Usar melt para cambiar el formato del DataFrame
    df = df.melt(var_name='label', value_name='intervalo')

    # Separar la columna 'intervalo' en 'inicio' y 'fin'
    df[['inicio', 'fin']] = df['intervalo'].str.split('-', expand=True)

    # Eliminar la columna 'intervalo'
    df.drop(columns='intervalo', inplace=True)

    # Reordenar las columnas
    df = df[['inicio', 'fin', 'label']]

    df['source'] = source
    df['subject'] = subject
    df = df.dropna(axis=0)
    dfs_concat.append(df)

In [7]:
IMG_df = pd.concat(dfs_concat)
IMG_df.to_csv('./dataframes/IMG_df.csv') #Se guarda en un archivo .csv para posterior analisis

In [9]:
# Crear la carpeta para guardar los videos recortados
output_folder = './sources'
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

# Iterar sobre cada fila del DataFrame
for index, row in IMG_df.iterrows():
    subject = row['subject']
    inicio = row['inicio']
    fin = row['fin']
    label = row['label']
    source = row['source']
    
    # Cargar el video
    video_capture = cv2.VideoCapture(source)
    
    # Obtener la tasa de fotogramas del video
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    
    # Verificar que el formato de inicio y fin sea válido
    if ':' not in inicio or ':' not in fin:
        print(f"Error en el formato de tiempo en la fila {index}. Saltando esta fila.")
        continue
    
    # Convertir los tiempos de inicio y fin a segundos
    inicio_segundos = int(inicio.split(':')[0]) * 60 + int(inicio.split(':')[1])
    fin_segundos = int(fin.split(':')[0]) * 60 + int(fin.split(':')[1])
    
    # Crear el nombre del archivo de salida
    folder_PAV = f'./sources/{label}'
    if not os.path.exists(folder_PAV):
        os.makedirs(folder_PAV)
    output_filename = f"{folder_PAV}/{subject}_{index}.mp4"
    
    # Crear un objeto VideoWriter para guardar el video recortado
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_filename, fourcc, fps, (int(video_capture.get(3)), int(video_capture.get(4))))
    
    # Establecer el cuadro actual en el tiempo de inicio
    video_capture.set(cv2.CAP_PROP_POS_FRAMES, inicio_segundos * fps)
    
    # Leer y escribir cada cuadro hasta el tiempo de finalización
    while video_capture.get(cv2.CAP_PROP_POS_FRAMES) < fin_segundos * fps:
        ret, frame = video_capture.read()
        if ret:
            video_writer.write(frame)
        else:
            break
    
    # Liberar los recursos
    video_capture.release()
    video_writer.release()

print("Videos recortados y guardados en la carpeta 'sources'.")

Videos recortados y guardados en la carpeta 'sources'.


### Analisis exploratorio

In [8]:
IMG_df = pd.read_csv('./dataframes/IMG_df.csv')
IMG_df = IMG_df[['inicio',	'fin',	'label',	'source',	'subject',]]
IMG_df

,inicio,fin,label,source,subject
0,00:00,00:12,FRENTE,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT
1,00:13,00:24,IZQUIERDA,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT
2,00:25,00:35,RETROVISOR,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT
3,00:37,00:47,DERECHA,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT
4,00:48,01:00,PALANCA,./Imagen/Scarlett/SCARLETT.mp4,SCARLETT
5,00:00,00:08,FRENTE,./Imagen/Andres/ANDRES.mp4,ANDRES
6,00:14,00:16,FRENTE,./Imagen/Andres/ANDRES.mp4,ANDRES
7,00:22,00:23,FRENTE,./Imagen/Andres/ANDRES.mp4,ANDRES
8,00:29,00:36,FRENTE,./Imagen/Andres/ANDRES.mp4,ANDRES
9,00:24,00:28,RETROVISOR,./Imagen/Andres/ANDRES.mp4,ANDRES


### Extracción de caracteristicas

### Selección de caracteristicas

### Entrenamiento de los modelos

### Implementación del modelo en tiempo real. 